## SegCaps Experiments
This notebook was used in the development of the capsule network models and investigation of their performance.

In [20]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import tensorflow as tf
from keras import layers, models
import numpy as np
import datetime
#import capsnet

import dataset
import model_functions as mf
import audio_functions as af
import audio_models

import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['image.cmap'] = 'hot'
import importlib

In [21]:
importlib.reload(audio_models)

<module 'audio_models' from 'C:\\Users\\Toby\\MSc_Project\\MagPhaseMask\\audio_models.py'>

In [25]:
#  Set variables
sample_rate=16384
n_fft=1024
fft_hop=256
patch_window=256
patch_hop=128
n_parallel_readers=16
normalise=True
batch_size = 1
shuffle=False
n_shuffle = 1
mag_phase = True
learning_rate = 0.0002
model_variant = 'unet'
data_type = 'mag_phase'
phase_weight = 0.005

checkpoint = '300/300-8'
model_base_dir = '/home/enterprise.internal.city.ac.uk/acvn728/checkpoints'

#directory_a = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Mixed'
#directory_b = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Voice'
#directory_c = '/home/enterprise.internal.city.ac.uk/acvn728/miniCHiME/Background'

directory_a = 'C:/Users/Toby/MSc_Project/Test_Audio/miniCHiME/Mixed'
directory_b = 'C:/Users/Toby/MSc_Project/Test_Audio/miniCHiME/Voice'
directory_c = 'C:/Users/Toby/MSc_Project/Test_Audio/miniCHiME/Background'


#  Create the pipeline
tf.reset_default_graph()
data = dataset.zip_files(directory_a, directory_b, directory_c)
data = dataset.get_paired_dataset(data,
                                  sample_rate,
                                  n_fft,
                                  fft_hop,
                                  patch_window,
                                  patch_hop,
                                  n_parallel_readers,
                                  batch_size,
                                  n_shuffle,
                                  normalise)

#  Create the iterator
pipe = data.make_initializable_iterator()
mixed_spec, voice_spec, background_spec, mixed_audio, voice_audio, background_audio = pipe.get_next()

#  Create variable placeholders
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_phase = tf.expand_dims(mixed_spec[:, :, :-1, 3], 3)
if data_type == 'mag':
    mixed_input = tf.expand_dims(mixed_spec[:, :, :-1, 2], 3)
    voice_input = tf.expand_dims(voice_spec[:, :, :-1, 2], 3)
elif data_type in ['mag_phase', 'mag_phase_diff']:
    mixed_input = mixed_spec[:, :, :-1, 2:4]
    voice_input = voice_spec[:, :, :-1, 2:4]
elif data_type == 'real_imag':
    mixed_input = mixed_spec[:, :, :-1, 0:2]
    voice_input = voice_spec[:, :, :-1, 0:2]
elif data_type in ['mag_real_imag', 'mag_phase2']:
    mixed_input = tf.concat([tf.expand_dims(mixed_spec[:, :, :-1, 2], 3), mixed_spec[:, :, :-1, 0:2]], 3)
    voice_input = tf.concat([tf.expand_dims(voice_spec[:, :, :-1, 2], 3), voice_spec[:, :, :-1, 0:2]], 3)
elif data_type in ['mag_phase_real_imag', 'complex_to_mag_phase']:
    mixed_input = mixed_spec[:, :, :-1, :]
    voice_input = voice_spec[:, :, :-1, :]
    
data

<PrefetchDataset shapes: ((?, 256, 513, 4), (?, 256, 513, 4), (?, 256, 513, 4), (?, 65280, 1), (?, 65280, 1), (?, 65280, 1)), types: (tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32)>

In [26]:
# Build U-Net model
print('Creating model')
model = audio_models.MagnitudeModel(mixed_input, voice_input, mixed_phase, mixed_audio, 
                                    voice_audio, background_audio, model_variant, is_training, learning_rate, 
                                    data_type, phase_weight, name='Magnitude_Model')
#tf_config = tf.ConfigProto()
#tf_config.gpu_options.allow_growth = True
#tf_config.gpu_options.visible_device_list = str(1)
#sess = tf.Session(config=tf_config)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

Creating model


In [27]:
sess.run(pipe.initializer)
print('start: {}'.format(datetime.datetime.now()))
epochs = 100
epoch = 0
epoch_cost = []
epoch_mag_loss = []
epoch_phase_loss = []
i = 0
while epoch < epochs:
    try:
        _, cost, mag_loss, phase_loss = sess.run([model.train_op, model.cost, model.mag_loss, model.phase_loss],
                                                {model.is_training: True})

        #print('Iteration {i}: {dt}\nCost: {c}'.format(i=i, dt=datetime.datetime.now(), c=cost))
        epoch_cost.append(cost)
        epoch_mag_loss.append(mag_loss)
        epoch_phase_loss.append(phase_loss)
        i += 1
    except tf.errors.OutOfRangeError:
        epoch_mean_cost = sum(epoch_cost)/len(epoch_cost)
        epoch_mean_mag_loss = sum(epoch_mag_loss) / len(epoch_mag_loss)
        epoch_mean_phase_loss = sum(epoch_phase_loss) / len(epoch_phase_loss)
        print('Epoch {e} finished. Mean cost = {emc}'.format(e=epoch, emc=epoch_mean_cost))
        print('\tMag loss: {emml}\n\tPhase loss: {empl}'.format(emml=epoch_mean_mag_loss, empl=epoch_mean_phase_loss))
        epoch += 1
        sess.run(pipe.initializer)
print('finish: {}'.format(datetime.datetime.now()))

start: 2019-01-21 17:09:55.132136
Epoch 0 finished. Mean cost = 0.0062892199493944645
	Mag loss: 0.005910462699830532
	Phase loss: 0.006667977198958397
Epoch 1 finished. Mean cost = 0.006274774204939604
	Mag loss: 0.005887962877750397
	Phase loss: 0.006661585532128811
Epoch 2 finished. Mean cost = 0.00626179048170646
	Mag loss: 0.0058681536465883255
	Phase loss: 0.006655427472045024
Epoch 3 finished. Mean cost = 0.006248936289921403
	Mag loss: 0.0058485716581344604
	Phase loss: 0.006649300921708345
Epoch 4 finished. Mean cost = 0.00623669233173132
	Mag loss: 0.005830006860196591
	Phase loss: 0.006643377896398306
Epoch 5 finished. Mean cost = 0.006223954570790132
	Mag loss: 0.005810724260906379
	Phase loss: 0.006637184880673885
Epoch 6 finished. Mean cost = 0.006210664553301675
	Mag loss: 0.005790398655725377
	Phase loss: 0.006630930450877973
Epoch 7 finished. Mean cost = 0.006196473084855825
	Mag loss: 0.005768225120846182
	Phase loss: 0.006624721048865467
Epoch 8 finished. Mean cost =

Epoch 70 finished. Mean cost = 0.004850020081224576
	Mag loss: 0.0033296732933745837
	Phase loss: 0.006370366839560824
Epoch 71 finished. Mean cost = 0.004837705268679808
	Mag loss: 0.0033080363516799277
	Phase loss: 0.006367374153342098
Epoch 72 finished. Mean cost = 0.0048256573484164395
	Mag loss: 0.00328694365293502
	Phase loss: 0.006364371015192711
Epoch 73 finished. Mean cost = 0.004813817380046523
	Mag loss: 0.00326619012205786
	Phase loss: 0.006361444606571584
Epoch 74 finished. Mean cost = 0.0048022103433807695
	Mag loss: 0.003245900298158328
	Phase loss: 0.0063585203637679414
Epoch 75 finished. Mean cost = 0.004790790936615514
	Mag loss: 0.0032260131280820226
	Phase loss: 0.006355568717576955
Epoch 76 finished. Mean cost = 0.004779514771970836
	Mag loss: 0.003206361151699509
	Phase loss: 0.006352668362004417
Epoch 77 finished. Mean cost = 0.004768433455282297
	Mag loss: 0.0031870912599305692
	Phase loss: 0.00634977562376895
Epoch 78 finished. Mean cost = 0.004757571040027881


start: 2019-01-21 17:05:15.660059
Epoch 0 finished. Mean cost = 0.006816165521740913
	Mag loss: 0.005709200631827116
	Phase loss: 0.007923129945993423
Epoch 1 finished. Mean cost = 0.006766058038920164
	Mag loss: 0.005687034921720624
	Phase loss: 0.00784508092328906
Epoch 2 finished. Mean cost = 0.006713496055454016
	Mag loss: 0.0056584190266827745
	Phase loss: 0.007768572929004828
Epoch 3 finished. Mean cost = 0.006658522295765579
	Mag loss: 0.0056322034215554595
	Phase loss: 0.007684841169975698
Epoch 4 finished. Mean cost = 0.006602700147777796
	Mag loss: 0.005605342797935009
	Phase loss: 0.007600057404488325
Epoch 5 finished. Mean cost = 0.006543238569671909
	Mag loss: 0.005575183779001236
	Phase loss: 0.0075112932827323675
Epoch 6 finished. Mean cost = 0.006481033284217119
	Mag loss: 0.005542839345123086
	Phase loss: 0.007419227223311152
Epoch 7 finished. Mean cost = 0.006416231277398765
	Mag loss: 0.005509370239451528
	Phase loss: 0.007323092315346003
Epoch 8 finished. Mean cost 

Epoch 70 finished. Mean cost = 0.0038861811233185965
	Mag loss: 0.004265436614063424
	Phase loss: 0.003506925655528903
Epoch 71 finished. Mean cost = 0.0038705689528594827
	Mag loss: 0.004256147930088143
	Phase loss: 0.0034849900015008948
Epoch 72 finished. Mean cost = 0.003855165735218827
	Mag loss: 0.004246734172003727
	Phase loss: 0.003463597320760154
Epoch 73 finished. Mean cost = 0.003839913522824645
	Mag loss: 0.004237304642997883
	Phase loss: 0.003442522426249107
Epoch 74 finished. Mean cost = 0.0038248460739851
	Mag loss: 0.004228051727016767
	Phase loss: 0.0034216404473409057
Epoch 75 finished. Mean cost = 0.003809951019081238
	Mag loss: 0.004218405945912788
	Phase loss: 0.0034014961182899576
Epoch 76 finished. Mean cost = 0.0037952108526384675
	Mag loss: 0.004209033648979354
	Phase loss: 0.0033813880850234396
Epoch 77 finished. Mean cost = 0.0037806285246729087
	Mag loss: 0.004199536618752739
	Phase loss: 0.0033617204589506565
Epoch 78 finished. Mean cost = 0.0037661747199400